In [1]:
# Imports
import pandas as pd
import numpy as np
import gzip
import time
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

# Sklearn libraries.
from sklearn import datasets, linear_model, ensemble, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import nltk
from nltk.corpus import stopwords
import string

from collections import Counter
#deep learning library

import json, os, re, shutil, sys, time
import collections, itertools
import unittest
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# utils.pretty_print_matrix uses Pandas. Configure float format here.
import pandas as pd
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# Helper libraries
from shared_lib import utils, vocabulary, tf_embed_viz




### 1. Load Data

In [2]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [3]:
# Unzip gz file.
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

# Load JSON into dataframe.
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')


In [4]:
start = time.time()
df = getDF('/home/legu/data/reviews_Clothing_Shoes_and_Jewelry_5.json.gz')
end = time.time()
print "Time Used %fs" %(end - start)

Time Used 42.877285s


### 2. Data Exploration

In [5]:
print df.columns

print df['reviewText'][0]

df.head(5)


Index([u'reviewerID', u'asin', u'reviewerName', u'helpful', u'unixReviewTime',
       u'reviewText', u'overall', u'reviewTime', u'summary'],
      dtype='object')
This is a great tutu and at a really great price. It doesn't look cheap at all. I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly. A++


,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall,reviewTime,summary
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",1297468800,This is a great tutu and at a really great pri...,5.0000,"02 12, 2011",Great tutu- not cheaply made
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",1358553600,I bought this for my 4 yr old daughter for dan...,5.0000,"01 19, 2013",Very Cute!!
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",1357257600,What can I say... my daughters have it in oran...,5.0000,"01 4, 2013",I have buy more than one
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]",1398556800,"We bought several tutus at once, and they are ...",5.0000,"04 27, 2014","Adorable, Sturdy"
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",1394841600,Thank you Halo Heaven great product for Little...,5.0000,"03 15, 2014",Grammy's Angels Love it


#### Exploring helpfulness scores.

In [6]:
helpfulness = []
total_votes = []
for i in df['helpful']:
    if i[1] == 0:
        helpfulness.append(np.nan)
        total_votes.append(np.nan)
    else:
        helpfulness.append(float(i[0])/i[1])
        total_votes.append(i[1])
        
# Convert to numpy array.
helpfulness = np.array(helpfulness)
total_votes = np.array(total_votes)

In [7]:
# Plot boxplot.
nonnan_helpfulness = helpfulness[~np.isnan(helpfulness)]


#### Exploring review text lengths.

In [8]:
# Remove invalid and nan values.
helpfulness_clean = np.delete(helpfulness, 30730)

nonnan_helpfulness_clean = helpfulness_clean[~np.isnan(helpfulness_clean)]



#### Exploring review scores.

### 3. Build Baseline Model

In [170]:
#extract the review text
doc = np.array(df['reviewText'])

# Filter down to reviews with helpfulness scores.
# Remove item with invalid helpfulness score.
doc_clean = np.delete(doc, 30730)
nonnan_doc_clean = doc_clean[~np.isnan(helpfulness_clean)]
y = np.reshape(nonnan_helpfulness_clean,(-1,1))

### 4. Deep Learning Exploration

#### Build LSTM Model

#### 1.Split the dataset into traing and test 

In [171]:
len(nonnan_doc_clean )
msk = np.random.rand(nonnan_doc_clean.shape[0]) <= 0.7

X_train = nonnan_doc_clean[msk]
X_test = nonnan_doc_clean[~msk]
y_train = y[msk]
y_test = y[~msk]
X_sample = nonnan_doc_clean[0:10000]
y_sample = y[0:10000]

In [172]:
X_train[0]

'Full and well stitched.  This tutu is a beautiful purple color that looks just like the picture.  It looks just adorable on our little fairy.'

In [173]:
print (len(X_train))
print (len(y_sample))

60922
10000


#### 2.Convert the word into index

In [174]:
def get_tokens(docs):
    tokens =[]
    for i in docs:
        lowers = i.lower()
        #remove the punctuation using the character deletion step of translate
        no_punctuation = lowers.translate(None, string.punctuation)
        token = nltk.word_tokenize(no_punctuation)
        tokens.extend(token)
    return tokens

tokens = get_tokens(X_sample)

In [175]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [176]:
dic, rev_dict = build_dataset(tokens)

In [177]:
dic['limited']

2032

In [178]:
rev_dict[2102]

'ability'

In [179]:
X_sample_ids = []
for i in X_sample:
    lowers = i.lower()
    #remove the punctuation using the character deletion step of translate
    no_punctuation = lowers.translate(None, string.punctuation)
    token = nltk.word_tokenize(no_punctuation)
    j=[dic[w] for w in token]
    X_sample_ids.append(j)
    

In [180]:
print (X_sample[0])
print (X_sample_ids[0])
print (y_sample[0])

For what I paid for two tutus is unbeatable anywhere!  I ordered a pink and turquios and they are vibrant and beautiful! The tutu is very full! Princess style! Not cheaply made! Not cheap materia! Obviously someone made these with love and care! I paid less than 7 bucks for a tutu I and I feel proud of my self for researching to the point of finding gold!Recommend 2-6 years!My daughter is two ! Wears size 4t and this skirt ( one size ) fit perfect and will probaly be able to accommodate her quickly growing waist for some time!
[8, 83, 1, 609, 8, 130, 10316, 6, 6286, 996, 1, 96, 3, 686, 2, 15274, 2, 12, 13, 2554, 2, 355, 0, 9136, 6, 26, 404, 3355, 224, 19, 1646, 87, 19, 285, 24819, 1211, 502, 87, 17, 20, 78, 2, 540, 1, 609, 282, 52, 468, 1172, 8, 3, 9136, 1, 2, 1, 117, 4277, 7, 11, 1648, 8, 4646, 4, 0, 591, 7, 767, 24030, 3689, 17126, 519, 6, 130, 424, 29, 8633, 2, 10, 1584, 33, 29, 36, 110, 2, 41, 22416, 27, 349, 4, 1772, 250, 504, 3146, 195, 8, 81, 57]
[ 0.875]


#### 3.build tensor flow model - training

In [181]:
train_df = pd.DataFrame(np.column_stack([X_sample_ids,y_sample]), 
                               columns=['list_words', 'label'])

In [182]:
train_df.head()

,list_words,label
0,"[8, 83, 1, 609, 8, 130, 10316, 6, 6286, 996, 1...",0.8750
1,"[404, 2, 48, 2067, 10, 9136, 6, 3, 355, 1280, ...",1.0000
2,"[3668, 1243, 2, 5, 6, 401, 23, 1110, 0, 76, 48...",0.5000
3,"[0, 971, 1, 636, 10, 11, 1867, 7963, 3, 970, 8...",1.0000
4,"[78, 10, 584, 304, 24, 48, 143, 697, 560, 2659...",0.0000


In [183]:
class SimpleDataIterator():
    def __init__(self, df):
        self.df = df
        self.size = len(self.df)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n-1 > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df.ix[self.cursor:self.cursor+n-1]
        self.cursor += n
        return res['list_words'], res['label'] 

In [184]:
data = SimpleDataIterator(train_df)
d = data.next_batch(3)
print(d[0])
print(d[1])

0    [38, 141, 31, 1337, 1098, 104, 4383, 4824, 193...
1    [12, 423, 26, 928, 43, 84, 2, 1437, 236, 3, 10...
2    [19, 82, 21, 53, 187, 64, 3, 496, 15, 17, 13, ...
Name: list_words, dtype: object
0   1.0000
1   1.0000
2   0.0000
Name: label, dtype: object


In [185]:
class PaddedDataIterator(SimpleDataIterator):
    def next_batch(self, n,max_len):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
#             self.max_len = max_len
        res = self.df.ix[self.cursor:self.cursor+n-1]
        self.cursor += n

        # Pad sequences with 0s so they are all the same length
        maxlen = max_len
        x = np.zeros([n, maxlen], dtype=np.int32)
        for i, x_i in enumerate(x):
            l=len(res['list_words'].values[i]) ##list length
            if l>maxlen: 
                x_i[:maxlen] = res['list_words'].values[i][:max_len]
            else:
                x_i[:l] = res['list_words'].values[i][:l]

        return x, res['label'] 

In [186]:
data_pad = PaddedDataIterator(train_df)
d = data_pad.next_batch(4,100)
print(d[1]) 

0   1.0000
1   1.0000
2   1.0000
3   0.5000
Name: label, dtype: object


In [187]:
batch_x, batch_y = PaddedDataIterator(train_df).next_batch(4,100)

In [188]:
batch_x

array([[ 1341,    10,   387,     6,    26,   673,     1,    65,  3071,
            2,     3,    29,  6988,  1590,     1,    65,   448,    29,
         1559,     2,  1071,     9,   195,     1,   510,     5,     0,
           76,   221,     6,     0,  1590,   494,     6,   121,     4,
          120,     8,    45,    72,  2201,     5,   232,    45,   117,
           38,     1,    67,    19,    31,     5,    71,     5,   188,
           36,     0,  1320,    24,    34,    21,    13,   546,     2,
           18,  1901, 20903,    10,    41,    69,    38,    16,    21,
           72,  2201,     5,   315,    36,   124,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [   25,     3,    77,   440,    20,    10,   432,   106,     7,
           39,     5,     6,   156, 30112,     1,    25,   125,     8,
            3,   329,   432,   369,     1,   204,     7,    6

In [189]:
batch_y.reshape([-1,1])
1-batch_y.reshape([-1,1])

np.concatenate((batch_y.reshape([-1,1]), 1-batch_y.reshape([-1,1])), axis=1)


/home/legu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
/home/legu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app
/home/legu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


array([[0.6666666666666666, 0.33333333333333337],
       [0.0, 1.0],
       [1.0, 0.0],
       [1.0, 0.0]], dtype=object)

In [190]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

# Import MNIST data


'''
To classify images using a recurrent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

# Parameters
learning_rate = 0.001
training_iters = 10000
batch_size = 100
display_step = 1

# Network Parameters
n_input = 10 # MNIST data input (img shape: 28*28)
n_steps = 10 # timesteps
n_hidden = 10 # hidden layer num of features
n_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)

    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    with tf.variable_scope("first_lstm29"):
        lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
  
        outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.nn.softmax(tf.matmul(outputs[-1], weights['out']) + biases['out'])

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y_1 = PaddedDataIterator(train_df).next_batch(batch_size,100)
        batch_y = np.concatenate((batch_y_1.reshape([-1,1]), 1-batch_y_1.reshape([-1,1])), axis=1)

        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
#             print( sess.run(pred, feed_dict={x: batch_x, y: batch_y}))
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
#     test_len = 128
#     test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
#     test_label = mnist.test.labels[:test_len]
#     print("Testing Accuracy:", \
#         sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

/home/legu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:89: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


Iter 100, Minibatch Loss= 0.943899, Training Accuracy= 0.17000
Iter 200, Minibatch Loss= 0.904176, Training Accuracy= 0.24000
Iter 300, Minibatch Loss= 0.912557, Training Accuracy= 0.22000
Iter 400, Minibatch Loss= 0.906996, Training Accuracy= 0.27000
Iter 500, Minibatch Loss= 0.899713, Training Accuracy= 0.21000
Iter 600, Minibatch Loss= 0.901354, Training Accuracy= 0.24000
Iter 700, Minibatch Loss= 0.884398, Training Accuracy= 0.28000
Iter 800, Minibatch Loss= 0.946870, Training Accuracy= 0.15000
Iter 900, Minibatch Loss= 0.893708, Training Accuracy= 0.23000
Iter 1000, Minibatch Loss= 0.907581, Training Accuracy= 0.20000
Iter 1100, Minibatch Loss= 0.851186, Training Accuracy= 0.23000
Iter 1200, Minibatch Loss= 0.906922, Training Accuracy= 0.20000
Iter 1300, Minibatch Loss= 0.896135, Training Accuracy= 0.21000
Iter 1400, Minibatch Loss= 0.907591, Training Accuracy= 0.20000
Iter 1500, Minibatch Loss= 0.875335, Training Accuracy= 0.22000
Iter 1600, Minibatch Loss= 0.863656, Training Acc

In [177]:
batch_x, batch_y = PaddedDataIterator(train_df).next_batch(batch_size,100)

In [159]:
batch_x

array([[   5,  176,   34,   33,   55,  473,    5,  371,   68,  111,    4,
        2523,  133,    5,   34,  371,    3,  140,  539,    2,   55,    1,
        2409,   22,    1,   64,  210,   20,  237,    5,  175,  335,   21,
           5,  193,  271,    5, 1097,  326,   53,    1,  216,   57,    9,
        1349,    1,   17,   10,  150,   15,  266,    4,  213,   31, 2480,
           9,  849,    1,   17,   15, 2183, 1508,  426,   11,   15,  352,
          13, 2397,  432,    4,    3,  179, 1624,   40,    0, 2087,  663,
           4,  891,  819,    2,  215,  192,   11,  512,    0,  266,  822,
           7,    3,   62,  207,   23,    1,   43,    2,  612,    5,  125,
          23],
       [   1,  182,   10,   91,    3,   62,  100,    3,  160,  329,  136,
          12,    1,   78,  451,   13,    5,    1,  263,    2,  968,    5,
           1,  124,    5,   16, 1273,    4, 1381,   36,   43,  131, 2129,
        1394,    0, 1723,    4,   17,  101,  692,    0, 1882,   11,    0,
        1361, 2415, 206

In [163]:
batch_x, batch_y = PaddedDataIterator(train_df).next_batch(batch_size,100)
batch_x = batch_x.reshape((batch_size, n_steps, n_input))

x = tf.unstack(x, n_steps, 1)

In [161]:
tf.unstack(x, n_steps, 1)

[<tf.Tensor 'unstack_5:0' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'unstack_5:1' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'unstack_5:2' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'unstack_5:3' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'unstack_5:4' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'unstack_5:5' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'unstack_5:6' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'unstack_5:7' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'unstack_5:8' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'unstack_5:9' shape=(?, 10) dtype=float32>]

In [156]:
from tensorflow.contrib import rnn

In [ ]:
##define variables 
n_hidden = 10 
with tf.variable_scope("softmax"):
    W_out_ = tf.Variable(tf.random_uniform([n_hidden,1], 0.0, 1.0), name="W_out_")
    b_out_ = tf.Variable(tf.zeros([1], dtype=tf.float32) ,name="b_out_")

with tf.variable_scope("recurrent"):
    rnn_cell = rnn.BasicLSTMCell(n_hidden)
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    initial_h_ = self.cell_.zero_state(self.batch_size_,dtype=tf.float32)
    outputs, self.final_h_ = tf.nn.dynamic_rnn(self.cell_, inputs=self.x_,initial_state=self.initial_h_, sequence_length=100)



In [196]:
def MakeFancyRNNCell(H, keep_prob, num_layers=1):
    """Make a fancy RNN cell.

    Use tf.nn.rnn_cell functions to construct an LSTM cell.
    Initialize forget_bias=0.0 for better training.

    Args:
      H: hidden state size
      keep_prob: dropout keep prob (same for input and output)
      num_layers: number of cell layers

    Returns:
      (tf.nn.rnn_cell.RNNCell) multi-layer LSTM cell with dropout
    """
    cells = []
    for _ in xrange(num_layers):
        cell = tf.contrib.rnn.BasicLSTMCell(H, forget_bias=0.0)
        cell = tf.contrib.rnn.DropoutWrapper(
        cell, input_keep_prob=keep_prob, output_keep_prob=keep_prob)
        cells.append(cell)
    return tf.contrib.rnn.MultiRNNCell(cells)


In [36]:
##have to fix here!!!
def RNN(x):
    n_hidden = 10 
    with tf.variable_scope("softmax"):
        W_out_ = tf.Variable(tf.random_uniform([n_hidden,1], 0.0, 1.0), name="W_out_")
        b_out_ = tf.Variable(tf.zeros([1], dtype=tf.float32) ,name="b_out_")
    # reshape to [1, n_input]
    n_input = len(x)
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)
#     print x
    # 1-layer LSTM with n_hidden units.
#     with tf.variable_scope('cell_def'):
#         lstm_cell = MakeFancyRNNCell(H=n_hidden,keep_prob=1)
    lstm_cell = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(n_hidden, forget_bias=0.0)
#     print x
    # generate prediction

    
#     with tf.variable_scope('rnn_def',reuse=True):
    initial_h_ = lstm_cell.zero_state(1,dtype=tf.int32)
    outputs,final_h_ = tf.contrib.rnn.static_rnn(lstm_cell, x,dtype=tf.int32)
            
            

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], W_out_) + b_out_

In [37]:
import tensorflow as tf
from tensorflow.contrib import rnn


learning_rate =0.1
pred = RNN(X_sample_ids[0])

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y_sample[0]))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

AttributeError: 'module' object has no attribute 'static_rnn'

In [ ]:
_, loss, onehot_pred = session.run([optimizer, cost, pred], 
                                        feed_dict={x: symbols_in_keys, y: symbols_out_onehot})

### Scratch below

In [17]:
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(60843,)
(26177,)
(60843, 1)
(26177, 1)


In [18]:
X_sample = nonnan_doc_clean[0:100]
y_sample = y[0:100]

In [19]:
print len(X_sample[0].split())
print X_sample[0]
print y[0]

35
This is a great tutu and at a really great price. It doesn't look cheap at all. I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly. A++
[ 0.875]


In [22]:
len(nltk.corpus.__getattr__('brown').words())

1161192

In [86]:
corpus = X_sample[0].split()
bi = utils.batch_generator(np.array(X_sample[0].split()), batch_size=1, max_time=100)

In [87]:
for i, (w,y) in enumerate(bi):
    print w
    print y_sample[0]

[['For' 'what' 'I' 'paid' 'for' 'two' 'tutus' 'is' 'unbeatable' 'anywhere!'
  'I' 'ordered' 'a' 'pink' 'and' 'turquios' 'and' 'they' 'are' 'vibrant'
  'and' 'beautiful!' 'The' 'tutu' 'is' 'very' 'full!' 'Princess' 'style!'
  'Not' 'cheaply' 'made!' 'Not' 'cheap' 'materia!' 'Obviously' 'someone'
  'made' 'these' 'with' 'love' 'and' 'care!' 'I' 'paid' 'less' 'than' '7'
  'bucks' 'for' 'a' 'tutu' 'I' 'and' 'I' 'feel' 'proud' 'of' 'my' 'self'
  'for' 'researching' 'to' 'the' 'point' 'of' 'finding' 'gold!Recommend'
  '2-6' 'years!My' 'daughter' 'is' 'two' '!' 'Wears' 'size' '4t' 'and'
  'this' 'skirt' '(' 'one' 'size' ')' 'fit' 'perfect' 'and' 'will'
  'probaly' 'be' 'able' 'to' 'accommodate' 'her' 'quickly' 'growing'
  'waist' 'for' 'some']]
[ 0.875]


In [99]:
import rnnlm
# import rnnlm_test
reload(rnnlm)
# reload(rnnlm_test)

<module 'rnnlm' from 'rnnlm.py'>

In [92]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=0.1):
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation

    for i, (w, y) in enumerate(batch_iterator):
        cost = 0.0
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})

        #### YOUR CODE HERE ####
        
        feed_dict = {lm.input_w_:w,
                 lm.target_y_:y,
                 lm.learning_rate_:learning_rate,
                 lm.initial_h_ : h}
            
        cost, _, h = session.run([loss, train_op,lm.final_h_],
                       feed_dict=feed_dict)
        
        
            
            
        #### END(YOUR CODE) ####
        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time

        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print "[batch %d]: seen %d words at %d wps, loss = %.3f" % (
                i, total_words, avg_wps, avg_cost)
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [93]:
V=1000
max_time = 20
batch_size = 50
learning_rate = 0.5
num_epochs = 5

# Model parameters
model_params = dict(V=100, 
                    H=100, 
                    softmax_ns=200,
                    num_layers=1)

# TF_SAVEDIR = "tf_saved"
# checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlm")
# trained_filename = os.path.join(TF_SAVEDIR, "rnnlm_trained")

In [101]:
# Will print status every this many seconds
import rnnlm
# import rnnlm_test
reload(rnnlm)
print_interval = 5

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
# shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
# if not os.path.isdir(TF_SAVEDIR):
#     os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    for epoch in xrange(1,num_epochs+1):
        t0_epoch = time.time()
#         bi = utils.batch_generator(train_ids, batch_size, max_time)
        corpus = X_sample[0].split()
        bi = utils.batch_generator(np.array(X_sample[0].split()), batch_size=1, max_time=100)
        for i, (w,y) in enumerate(bi):
            w1=w
            
        bi= (w1,y_sample[y])
        print "[epoch %d] Starting epoch %d" % (epoch, epoch)
        #### YOUR CODE HERE ####
        # Run a training epoch.
        cost = run_epoch(lm, session, bi, 
                     learning_rate=learning_rate, train=True, 
                     verbose=False, tick_s=3600)

        #### END(YOUR CODE) ####
        print "[epoch %d] Completed in %s" % (epoch, utils.pretty_timedelta(since=t0_epoch))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        print ("[epoch %d]" % epoch),
        #score_dataset(lm, session, train_ids, name="Train set")
        print ("[epoch %d]" % epoch),
        score_dataset(lm, session, test_ids, name="Test set")
        print ""
    
    # Save final model
    saver.save(session, trained_filename)

ValueError: Dimension must be 2 but is 3 for 'recurrent/transpose' (op: 'Transpose') with input shapes: [?,100], [3].

## Scratch Below

In [ ]:
print 1

In [62]:
import nltk
nltk.download('brown')

corpus = nltk.corpus.brown

[nltk_data] Downloading package brown to /Users/legu/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [72]:
!pip install vocabulary

    100% |████████████████████████████████| 593kB 408kB/s 
    100% |████████████████████████████████| 61kB 1.9MB/s 
    100% |████████████████████████████████| 102kB 1.2MB/s 
  Running setup.py bdist_wheel for vocabulary ... - \ done
  Stored in directory: /Users/legu/Library/Caches/pip/wheels/36/6c/c0/92bb20f79402d055c3bce3e89d9f2cce5d6937bc2aadc0fb45
Successfully built vocabulary
  Found existing installation: requests 2.11.1
    Uninstalling requests-2.11.1:
      Successfully uninstalled requests-2.11.1
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [73]:
import utils
import vocabulary
# train_sents, test_sents = utils.get_train_test_sents(corpus, split=0.8, shuffle=True)
vocab = vocabulary.Vocabulary(utils.canonicalize_word(w) for w in utils.flatten(corpus))
print "Train set vocabulary: %d words" % vocab.size

AttributeError: 'module' object has no attribute 'Vocabulary'